# Autoregressive (AR)

### AR method models the next step in sequence as a linear function of observation at prior time steps.
AR(1) - First order auto-regressive model.
The method is suitable for univariate time series without trend and seasonal components.

An auto-regressive model is when a value from a time series is regressed on previous values from that time series.
eg. y(t) on y(t-1)

In [1]:
# Importing libraries
from statsmodels.tsa.ar_model import AR
from random import random

# Generation of dataset using random function
data = [x + random() for x in range(1,100)]

# Initialize and fit AR model
model = AR(data)
model_fit = model.fit()

# Predict the next value in the series
yhat = model_fit.predict(len(data), len(data))
print(yhat)

[100.49367857]


# Moving Average (MA)

### Rather than using past values of the forecast variable in a regression, a moving average model uses past forecast errors in a regression-like model.

The 1st order MA model can be described as :
x(t) = (mu) + w(t) + theta(1)*w(t-1)

It is different from calculating the moving average of the time series.
The method is suitable for univariate time series without trend and seasonal components.

In [3]:
from statsmodels.tsa.arima_model import ARMA
from random import random

data = [x + random() for x in range(1,100)]

model = ARMA(data, order=(0,1))
model_fit = model.fit(disp=False)

# Predict the next value in the series
yhat = model_fit.predict(len(data), len(data))
print(yhat)

[75.6351011]


In [4]:
data

[1.908421245492853,
 2.559611344025374,
 3.800027753676085,
 4.6789451541324665,
 5.595534972258833,
 6.381913403601008,
 7.002720610281833,
 8.22369865699575,
 9.089643200221808,
 10.857920771480199,
 11.33880563253665,
 12.624719929083033,
 13.878638671683152,
 14.67465324948628,
 15.527389785068022,
 16.01477645816657,
 17.43148241223358,
 18.16637470417844,
 19.62445109441028,
 20.6369811644463,
 21.24743169844654,
 22.790922690184928,
 23.91741034784298,
 24.764782332922696,
 25.29513113865987,
 26.235175538358313,
 27.85341724658559,
 28.461851984333457,
 29.895055362932457,
 30.5881173095709,
 31.927400268447574,
 32.49078582059991,
 33.348597956809634,
 34.8349810272693,
 35.38619024371969,
 36.14704601603171,
 37.92819606043505,
 38.80575964983762,
 39.89766609015618,
 40.63747893198025,
 41.928795439787635,
 42.692670890352254,
 43.15380202058246,
 44.858411573928265,
 45.06249287870993,
 46.12699887443495,
 47.3452102005473,
 48.56411335975722,
 49.65070779334203,
 50.641908